In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#REF: https://medium.com/@shubhama94262/building-a-multiple-choice-question-app-using-langchain-and-llm-model-d59839fd1150

In [1]:
!pip install openai
!pip install pinecone-client
!pip install langchain
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 6.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which i

In [3]:
!pip install -U langchain-community

In [4]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension enable --py --sys-prefix ipywidgets
!jupyter nbextension install --py widgetsnbextension --sys-prefix

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Traceback (most recent call last):
  File "/opt/conda/bin/jupyter-nbextension", line 10, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.10/site-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.10/site-packages/notebook/nbextensions.py", line 972, in start
    super().start()
  File "/opt/conda/lib/python3.10/site-packages/jupyter_core/application.py", line 270, in start
    self.subapp.start()
  File "/opt/conda/lib/python3.10/site-packages/notebook/nbextensions.py", line 882, in start
    self.toggl

In [5]:
# Import necessary packages
import openai
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [6]:
#Function to read documents
def load_docs(directory):
  loader = PyPDFDirectoryLoader(directory)
  documents = loader.load()
  return documents

# Passing the directory to the 'load_docs' function
directory = '/kaggle/input/ai-gov-terms'
documents = load_docs(directory)
len(documents)

13

In [7]:
#This function will split the documents into chunks
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

46


In [8]:

# Initialize embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Test our embeddings model for a sample text
query_result = embeddings.embed_query("Hello Buddy")
print(f"Length of query result: {len(query_result)}")
print(query_result)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
2024-08-08 02:25:32.023555: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 02:25:32.023691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 02:25:32.200295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS fact

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Length of query result: 384
[-0.06978833675384521, 0.054206255823373795, 0.07814784348011017, 0.03390119597315788, 0.024947457015514374, -0.0967373251914978, 0.059523146599531174, 0.05897815153002739, -0.017896685749292374, -0.023178884759545326, -0.019000209867954254, 0.0005968869081698358, 0.024666039273142815, -0.07030834257602692, -0.00752252945676446, 0.010224476456642151, -0.01118082832545042, -0.021248547360301018, -0.03859451785683632, 0.0265504140406847, -0.06505240499973297, 0.0650002732872963, 0.009431800805032253, -0.06271227449178696, -0.023625478148460388, -0.03063809685409069, 0.0599612295627594, 0.07367487996816635, -0.03286782652139664, -0.02606102265417576, -0.006967172026634216, 0.03061792254447937, 0.05939662456512451, 0.0014719939790666103, 0.012021698988974094, 0.028293700888752937, -0.05922519415616989, -0.07919753342866898, 0.04896368831396103, 0.023090122267603874, 0.05536279454827309, -0.026251379400491714, -0.01732114888727665, 0.005511110182851553, 0.0586276

In [2]:
!pip install pinecone-client langchain sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 18.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
   

In [5]:
!pip install -U langchain-community
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.9 MB/s eta 0:00:0000:010:01


In [23]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone

# Set your Pinecone API key and environment
api_key = "Pinecone_API_key"
environment = "us-east-1"  # e.g., "us-west1-gcp"

# Ensure the API key is set as an environment variable
os.environ["PINECONE_API_KEY"] = api_key

# Initialize Pinecone
pc = Pinecone(api_key=api_key)

# Define your index name
index_name = "mcq-creator"

# Delete the existing index if necessary (only if the dimension is incorrect)
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# Create a new index with the correct dimension
pc.create_index(
    name=index_name,
    dimension=384,  # Ensure this matches your embedding dimension
    metric='cosine',  # Choose the appropriate metric
    spec=ServerlessSpec(
        cloud="aws",  # Or 'aws'
        region= "us-east-1"  # e.g., 'us-west1-gcp'
    )
)

# Function to read documents
def load_docs(directory):
    loader = PyPDFDirectoryLoader(directory)
    documents = loader.load()
    return documents

# Update the path to your actual directory path
directory = '/kaggle/input/ai-gov-terms'  # Example for Windows
# directory = '/Users/YourUsername/Desktop/pdf_files/'  # Example for macOS/Linux

# Passing the directory to the 'load_docs' function
documents = load_docs(directory)
print(f"Number of documents loaded: {len(documents)}")

# Initialize embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create an instance of Pinecone from Langchain and populate the index
vector_store = LangchainPinecone.from_documents(documents, embeddings, index_name=index_name)

# Add documents to the index
vector_store.add_documents(documents)


Number of documents loaded: 13


['b3ebd37d-8a86-47ba-a112-1ef8518ba3bd',
 'c8cc2537-4b9e-4be7-8fed-2c2c91c34219',
 '1b04ab24-bd3f-4fe0-a0fb-421acaa9d189',
 'aa295f0e-9267-47c6-9fd2-5909326edab9',
 'fa460306-0a38-49b7-aec3-958d44fed3c3',
 '04e07488-e9af-41e3-a5f2-716773d434fd',
 'f4b47af9-6ee6-4245-905d-c7e0ec2d2598',
 '6c1bc073-bfa4-4346-a3c0-6f6db5135562',
 '9e533acd-b042-4a90-be41-e65411c1696e',
 '7981fd6b-06b8-456b-9d01-05e9b8804e16',
 '667cc791-db13-47ea-a074-33ee480fced5',
 '0e061bd8-de96-443a-a929-336cc13a25e7',
 '1a03fdad-3a1d-47be-8d8c-bd36fbbefc4a']

In [25]:
#This function will help us in fetching the top relevent documents from our vector store - Pinecone
def get_similiar_docs(query, k=2):
    similar_docs = index.similarity_search(query, k=k)
    return similar_docs

In [26]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [2]:
!pip install -U langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 8.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires 

In [10]:
import os
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import login

# Set your Hugging Face API token
huggingface_api_token = "Huggingface_API_Token"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_api_token

# Log in to Hugging Face and add the token to git credentials
login(token=huggingface_api_token, add_to_git_credential=True)

# Initialize the HuggingFaceEndpoint with explicit parameters
llm = HuggingFaceEndpoint(
    repo_id="bigscience/bloom",
    huggingfacehub_api_token=huggingface_api_token,
    temperature=1e-10  # Specify temperature explicitly
)

print(llm)


Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
HuggingFaceEndpoint
Params: {'endpoint_url': None, 'task': None, 'model_kwargs': {}}


In [14]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 11.4 MB/s eta 0:00:0000:010:01


In [18]:
from langchain.chains.question_answering import load_qa_chain

In [20]:
pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain.chains.question_answering import load_qa_chain

# Assuming `llm` is already defined as your language model
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
    return chain.run(query)


In [24]:
chain = load_qa_chain(llm, chain_type="stuff")
#This function will help us get the answer to the question that we raise
def get_answer(query):
  relevant_docs = get_similiar_docs(query)
  print(relevant_docs)
  response = chain.run(input_documents=relevant_docs, question=query)
  return response

In [29]:
def get_similiar_docs(query):
    # Implement your logic to retrieve relevant documents based on the query.
    # This is just a placeholder. Replace it with your actual code.
    return []

def get_answer(query):
    relevant_docs = get_similiar_docs(query)
    print(relevant_docs)
    response = chain.run(input_documents=relevant_docs, question=query)
    return response

# Now you can pass your question to the above created function
our_query = "How is India's economy?"
answer = get_answer(our_query)
print(answer)


[]


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 India is the world's largest democracy and the second most populous country in the world. It is a developing country with a large population of young people. It has a large and growing economy, but it is still a poor country. It is a member of the G20, the world's largest economies. It is a member of the Commonwealth of Nations, a group of former British colonies. It is a member of the United Nations and the World Trade Organization. It is a member of the South Asian Association for Regional Cooperation, a regional organization of South Asian countries. It is a member of the BRICS, a group of five major emerging economies. It is a member of the BASIC, a group of four major emerging economies. It is a member of the G-20, a group of major emerging economies. It is a member of the BASIC, a group of four major emerging economies. It is a member of the BASIC, a group of four major emerging economies. It is a member of the BASIC, a group of four major emerging economies. It is a member of t

In [32]:
!pip install -U langchain

In [34]:
!pip install -U langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.4 MB/s eta 0:00:0000:0100:01


In [35]:
# Structures the Output

import re
import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [37]:
response_schemas = [
    ResponseSchema(name="question", description="Question generated from provided input text data."),
    ResponseSchema(name="choices", description="Available options for a multiple-choice question in comma separated."),
    ResponseSchema(name="answer", description="Correct answer for the asked question.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='question', description='Question generated from provided input text data.', type='string'), ResponseSchema(name='choices', description='Available options for a multiple-choice question in comma separated.', type='string'), ResponseSchema(name='answer', description='Correct answer for the asked question.', type='string')])

In [38]:
output_parser
# This helps us fetch the instructions the langchain creates to fetch the response in desired format
format_instructions = output_parser.get_format_instructions()
 
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"question": string  // Question generated from provided input text data.
	"choices": string  // Available options for a multiple-choice question in comma separated.
	"answer": string  // Correct answer for the asked question.
}
```


In [40]:
import os
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

In [42]:
!pip install openai
chat_model = ChatOpenAI(openai_api_key="your_openai_api_key_here")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 6.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 whic

In [43]:
# create ChatGPT object
chat_model = ChatOpenAI()

# The below snippet will give out a string that contains instructions for how the response should be formatted, and we then insert that into our prompt.

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("""When a text input is given by the user, please generate multiple choice questions 
        from it along with the correct answer. 
        \n{format_instructions}\n{user_prompt}""")  
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)
final_query = prompt.format_prompt(user_prompt = answer)
print(final_query)

messages=[HumanMessage(content='When a text input is given by the user, please generate multiple choice questions \n        from it along with the correct answer. \n        \nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"question": string  // Question generated from provided input text data.\n\t"choices": string  // Available options for a multiple-choice question in comma separated.\n\t"answer": string  // Correct answer for the asked question.\n}\n```\n India is the world\'s largest democracy and the second most populous country in the world. It is a developing country with a large population of young people. It has a large and growing economy, but it is still a poor country. It is a member of the G20, the world\'s largest economies. It is a member of the Commonwealth of Nations, a group of former British colonies. It is a member of the United Nations and the World Trade Organi

In [47]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:0000:0100:01


In [55]:
!pip install openai

In [56]:
from openai import OpenAI

In [57]:
import openai
print(openai.__version__)

1.40.1


In [61]:
import re
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

# Set your OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = "OpenAI_API_KEY"

# Create ChatGPT object
chat_model = ChatOpenAI()

# Prepare prompt
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("""When a text input is given by the user, please generate multiple choice questions 
        from it along with the correct answer. 
        \n{format_instructions}\n{user_prompt}""")  
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": "Your format instructions here."}
)
final_query = prompt.format_prompt(user_prompt="Your input text here")

# Get model response
final_query_output = chat_model(final_query)

# Extract JSON data from Markdown text
markdown_text = final_query_output.content
json_string = re.search(r'{(.*?)}', markdown_text, re.DOTALL).group(1)
print(json_string)



RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}